### Lazy Propagation
- Segment Tree에서 노드의 업데이트를 미루다가, 쿼리를 받으면 그때 업데이트를 하는 방식
- 각 노드에 대응하는 lazy 배열을 두어, 업데이트를 미룬다.
  - 해당 노드가 관장하는 구간 전체에 대해서 어떤 연산을 미루고 있는지에 대한 정보가 들어있다.
    - 구간합 세그의 경우 얼마를 더할 것인지, xor 세그의 경우 부호를 바꿀지의 여부 등
- 쿼리를 받으면, 해당 노드의 lazy를 업데이트하고, 자식 노드에게 propagate한다.
- 길이 $N$인 구간 $[a, b]$의 모든 인덱스에 연산을 적용하는 쿼리 $Q$ 의 구현을 Naive하게 적용한다면 그 구간의 길이 $b-a = K$ 라고 했을 때 $O(K \log N)$ 이 걸린다.\
하지만 Lazy Propagation을 적용하면 $O(\log N)$ 으로 줄일 수 있다.

#### Propagate
- 완전히 포함되면 lazy만 매기고, propagate한다

### 구현(재귀)
- query의 내부를 수정하기 쉬운 템플릿
- 코드 주석은 10999(구간 합 구하기 2) 참고

In [ ]:
class LST:
  def __init__(self, data):
    self.size = 1 << (len(data) - 1).bit_length()
    self.L = [0] * (2 * self.size)
    self.lazy = [0] * (2 * self.size)
    self.L[self.size:self.size + len(data)] = data
    for i in reversed(range(self.size)):
      self.L[i] = self.L[i*2] + self.L[i*2+1]

  def push(self, i, s, e) :
    if not self.lazy[i] : return

    self.L[i] += (e - s) * self.lazy[i]
    if i < self.size :
      self.lazy[i*2] += self.lazy[i]
      self.lazy[i*2+1] += self.lazy[i]
    self.lazy[i] = 0

  def query(self, l, r):
    return self._query(l, r, 1, 0, self.size)

  def _query(self, l, r, i, nl, nr) :
    self.push(i, nl, nr)
    
    if r <= nl or nr <= l : return 0
    if l <= nl and nr <= r : return self.L[i]
    mid = (nl + nr) // 2
    return self._query(l, r, i*2, nl, mid) + self._query(l, r, i*2+1, mid, nr)
  
  def update(self, l, r, x):
    self._update(l, r, x, 1, 0, self.size)

  def _update(self, l, r, x, i, nl, nr) :
    self.push(i, nl, nr)

    if r <= nl or nr <= l : return
    if l <= nl and nr <= r :
      self.lazy[i] += x
      self.push(i, nl, nr)
      return
  
    mid = (nl + nr) // 2
    self._update(l, r, x, i*2, nl, mid)
    self._update(l, r, x, i*2+1, mid, nr)
    self.L[i] = self.L[i*2] + self.L[i*2+1]

### iterative 구현
- update와 query 이전에 구간에 대한 propagate가 진행된다.
- apply는 직계 자식에게 push
- build는 모든 조상에 대해, propgate는 모든 자식에게 쿼리에 대한 정보를 알린다.
- 쿼리할 때 들어가는 연산 `x`는 `LazyNode`과 자료구조가 같다.
- f(a: `Node`, b: `Node`) -> `Node`
  - 노드와 노드끼리의 연산.
  - 세그먼트 트리를 초기화 할 때에도 노드끼리 이 연산이 사용된다.
  - 기본적으로, 쿼리의 해답을 구하기 위해 필요한 연산이다.
- fl(v: `Node`, x: `LazyNode`, cnt: `int`) -> `Node`
  - 노드 `v`에 대해 연산 `x` 서브노드의 개수 `cnt`번 반복한 것과 같은 값을 적용시킨다.
  - 리프 노드에 대해서만 생각하지 말고, 리프 노드가 아닌 노드에 대해서도 생각해야 한다. 
  - x가 cnt회 반복될 때의 특성을 잘 이해해야 한다.
    - 특히, xor연산 등을 할 땐 `cnt - v` 의 표현을 자주 쓰게 될 것이다.
- fll(v: `LazyNode`, x: `LazyNode`) -> `LazyNode`
  - 레이지 노드 `v` 에 대해 연산 `x`를 적용시킨 `LazyNode`를 반환한다.
  - 리프 노드가 아닌 lazy끼리의 연산
  - 연산 순서는 `v < x` 이다. 즉 피연산자가 `v`이고, 연산자는 `x` 이다.
    - 어떤 수를 대입해야 하는것이 연산이라면, `v`의 대입해야 하는 값은 무시되고, `x`의 대입값을 따르면 된다.
- `for i in range(N) : lst._apply(i)` seg 값을 디버그해야 한다면 이 코드를 수행시킨 후에 해야한다.
  - 쿼리를 직접 짜야 할 때도 마찬가지인데, 보려는 노드들은 전부 _apply(i)를 적용해야 lazy에 있는 값이 노드에 적용된다.
  - 자료구조 테크닉의 MEX참고

#### f, fl, fll 템플릿
- 구간합일 경우

In [ ]:
class F :
  def __init__(self) :
    self.f = lambda x, y : x + y
    self.fl = lambda i, x, cnt: i + x * cnt
    self.fll = lambda i, x: i + x

- 구간 max

In [ ]:
class F :
  def __init__(self) :
    self.f = max
    self.fl = lambda i, x, cnt: i + x
    self.fll = lambda i, x: i + x

In [ ]:
class F :
  def __init__(self) :
    self.f = lambda x, y : x + y
    self.fl = lambda i, x, cnt: i + x * cnt
    self.fll = lambda i, x: i + x

class LST(F):
  def __init__(self, L, node=0, lnode=0):
    F.__init__(self)
    self.node = node
    self.lnode = lnode
    self.len = len(L)
    self.size = size = 1 << (self.len - 1).bit_length()

    self.seg = [node] * (2 * size)
    self.seg[size:size + self.len] = L
    self.cnt = [0] * self.size + [1] * self.len + [0] * (self.size - self.len)
    self.lazy = [lnode] * size

    for i in reversed(range(self.size)):
      self.seg[i] = self.f(self.seg[i*2], self.seg[i*2+1])
      self.cnt[i] = self.cnt[i*2] + self.cnt[i*2+1]

  def _push(self, i, x) :
    self.seg[i] = self.fl(self.seg[i], x, self.cnt[i])
    if i < self.size :
      self.lazy[i] = self.fll(self.lazy[i], x)

  def _apply(self, i) :
    if self.lazy[i] == self.lnode: return
    self._push(i*2, self.lazy[i])
    self._push(i*2+1, self.lazy[i])
    self.lazy[i] = self.lnode 

  def _propagate(self, i):
    for h in reversed(range(1, self.size.bit_length()+1)) :
      if idx := i >> h : self._apply(idx)

  def _build(self, i) :
    while i :
      if i < self.size :
        self._apply(i)
        self.seg[i] = self.f(self.seg[i*2], self.seg[i*2+1])
      i >>= 1

  def query(self, s, e) :
    s += self.size
    e += self.size
    self._propagate(s)
    self._propagate(e-1)

    res = self.node
    while s < e :
      if s & 1 :
        res = self.f(res, self.seg[s])
        s += 1
      if e & 1 :
        e -= 1
        res = self.f(res, self.seg[e])
      s >>= 1
      e >>= 1
    return res

  def update(self, s, e, x):
    s += self.size
    e += self.size
    self._propagate(s)
    self._propagate(e-1)
    _s = s
    _e = e

    while s < e :
      if s & 1 :
        self._push(s, x)
        s += 1
      if e & 1 :
        e -= 1
        self._push(e, x)
      s >>= 1
      e >>= 1
    
    self._build(_s)
    self._build(_e - 1)